Соединение и аутентификация

При удачном подключении к почте увидим сообщение [b'Authentication successful']

In [8]:
import imaplib
import email
import os
import re
from email.header import decode_header
from bs4 import BeautifulSoup

def read(sender_of_interest=None):
    # Очищаем файл перед записью
    with open('filtered_emails.txt', 'r+') as f:
        f.truncate(0) 

    # Читаем учетные данные из переменных окружения
    email_user = os.getenv('EMAIL_USER')
    email_pass = os.getenv('EMAIL_PASS')

    # Подключаемся к серверу
    imap = imaplib.IMAP4_SSL("imap.gmail.com", 993)
    # imap.login(email_user, email_pass)
    imap.login('remboplas@gmail.com', 'jmsv ehgo ncwi sgrj')
    imap.select('INBOX')

    # Поиск всех писем
    if sender_of_interest:
        status, response = imap.uid('search', None, 'ALL', f'FROM {sender_of_interest}')
    else:
        status, response = imap.uid('search', None, 'ALL')

    if status == 'OK':
        unread_msg_nums = response[0].split()
        print(f"Found {len(unread_msg_nums)} emails")  # Отладочная информация
    else:
        unread_msg_nums = []
        print("No emails found")

    # Инвертируем список ID, чтобы получить последние письма первыми
    unread_msg_nums = unread_msg_nums[::-1]

    data_list = []
    count = 0  # Счётчик писем

    for e_id in unread_msg_nums:
        if count >= 400:  # Ограничение на 400 последних писем
            break

        e_id = e_id.decode('utf-8')
        _, response = imap.uid('fetch', e_id, '(RFC822)')

        # Используем message_from_bytes для правильного парсинга писем
        raw_email = response[0][1]
        email_message = email.message_from_bytes(raw_email)

        # Проверка, multipart или нет, чтобы обработать текстовые и HTML части
        if email_message.is_multipart():
            for part in email_message.walk():
                content_type = part.get_content_type()
                if content_type == 'text/plain':  # Извлекаем только текстовую часть
                    body = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                    data_list.append(body)
                    print(f"Added text/plain part from email {e_id}")  # Отладочная информация
                    count += 1
                    break  # Если нашли нужную часть, выходим
        else:
            # Если не multipart, просто получаем тело письма
            body = email_message.get_payload(decode=True).decode('utf-8', errors='ignore')
            data_list.append(body)
            count += 1

    # Запись в файл
    with open('filtered_emails.txt', 'w', encoding='utf-8') as f:
        for elem in data_list:
            # Удаляем блоки стилей <style>...</style>
            elem = re.sub(r"<style.*?>.*?</style>", "", elem, flags=re.S)
            elem = re.sub(r"&nbsp;", "", elem, flags=re.S)
            # Удаляем все HTML-теги
            elem = re.sub(r"<[^>]+>", "", elem, flags=re.S)
            
            if 'Высылаю обратную связь' in elem:
                print(f"Added non-multipart email {e_id}")  # Отладочная информация

                # Разбиваем на строки и записываем строки, если они не пустые
                for line in elem.splitlines():
                    if line.strip():  # Проверяем, не пустая ли строка
                        f.write(line + '\n')  # Записываем в файл

                # Добавляем разделитель для каждого письма
                f.write('\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')

    # Завершаем работу с сервером
    imap.logout()

# Вызов функции
read()


Found 1833 emails
Added text/plain part from email 5120
Added text/plain part from email 5119
Added text/plain part from email 5117
Added text/plain part from email 5116
Added text/plain part from email 5113
Added text/plain part from email 5112
Added text/plain part from email 5111
Added text/plain part from email 5110
Added text/plain part from email 5109
Added text/plain part from email 5107
Added text/plain part from email 5102
Added text/plain part from email 5101
Added text/plain part from email 5100
Added text/plain part from email 5099
Added text/plain part from email 5098
Added text/plain part from email 5097
Added text/plain part from email 5096
Added text/plain part from email 5095
Added text/plain part from email 5094
Added text/plain part from email 5093
Added text/plain part from email 5092
Added text/plain part from email 5091
Added text/plain part from email 5090
Added text/plain part from email 5089
Added text/plain part from email 5088
Added text/plain part from email

In [9]:
with open('filtered_emails.txt', 'r') as file:
    lines = file.readlines()

text = []  # Список для хранения блоков текста
cnt = 0  # Начальная позиция для нарезки

# Проходим по всем строкам файла
for i in range(len(lines)):
    # Проверяем, если строка содержит только плюсики (ваш разделитель)
    if lines[i].strip() == '++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++':
        # Добавляем блок текста от cnt до i в список text
        text.append(''.join(lines[cnt:i]).strip())
        cnt = i + 1  # Обновляем начало следующего блока

# Добавляем последний блок текста, если он есть после последней строки с плюсами
if cnt < len(lines):
    text.append(''.join(lines[cnt:]).strip())

# print(text[0])
print(text[len(text)-1])


-------- Перенаправленное сообщение --------
Тема: 	ОС по итогам просмотра Мышанского Владимира Викторовича от 
26.09.2024.
Дата: 	Tue, 01 Oct 2024 17:57:40 +0300
От: 	Тернов Сергей 
Отвечать: 	Тернов Сергей 
Кому: 	Алексей Аникеев 
Копия: 	Сергей Викторович Лавриченко , Ваге Акобян 
Добрый день.
Высылаю обратную связь по продавцу Скляров Николай Евгеньевич , название 
ТТ г.Гуково, ул. Комсомольская, д.49 Б, дата: 27.09.2024
В указанный период была: Продажа
*1. Оценка*
Максимальная оценка проверок по критериям – 100 баллов. Итоги просмотра:
Уровень сервиса - 23 из 25
НЭП - 60 из 67
Продажа аксессуаров - 0 из 6
Отработка по акции - 2 из 2
Итоговая сумма баллов с учетом обязательных и не обязательных пунктов - 
85 балов или 85%
*2.Выявленные отклонения, рекомендация*
Продавец не в уставной форме обуви.
Продавец не проверил состояние ткущего АКБ.
Клиенту не было предложено доп. товары. Так же не было демонстрации 
аксессуаров.
Клинту не выдали визитных карточек и листовок.
С уважением, Те

In [10]:
import re
import pymorphy2

# Инициализация MorphAnalyzer
morph = pymorphy2.MorphAnalyzer()

# Функция для нормализации ФИО
def fio_converter(fio):
    words = fio.split()  # Разделяем ФИО на отдельные слова
    normalized_words = []
    
    for word in words:
        parsed = morph.parse(word)[0]  # Получаем наиболее вероятный разбор слова
        # Приводим слово к нормальной форме в мужском роде, именительном падеже, единственном числе
        if 'Name' in parsed.tag or 'Patr' in parsed.tag:
            inflected = parsed.inflect({'nomn', 'masc', 'sing'})
            word_normal = inflected.word if inflected else parsed.normal_form
        else:
            word_normal = parsed.normal_form
        normalized_words.append(word_normal.capitalize())
    
    return ' '.join(normalized_words)

# Функции для удаления текста по правилам
def sliser(string, substring, flag):
    index = string.find(substring)
    if index == -1:
        return string

    index_back_n = string.find('\n', index)
    if index_back_n == -1:
        index_back_n = len(string)

    if flag == 'delete_after':
        string = string[:index].strip()
    elif flag == 'delete_before':
        string = string[index_back_n + 1:].strip()
    elif flag == 'delete_only':
        string = string[:index].strip() + ' ' + string[index_back_n + 1:].strip()
    return string

def multi_sliser(string, substrings, flag):
    for substring in substrings:
        string = sliser(string, substring, flag)
    return string

# Удаление стоп-слов, кроме цифр
def remove_stopwords(text, stopwords):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords or word.isdigit()]
    return ' '.join(filtered_words)

# Удаление повторяющихся слов, сохраняя их первый экземпляр
def remove_duplicates(text):
    seen = set()
    unique_words = []
    for word in text.split():
        if word not in seen:
            seen.add(word)
            unique_words.append(word)
    return ' '.join(unique_words)

# Функция для удаления всех ">" в начале строки
def remove_leading_arrows(text):
    # Регулярное выражение удалит все символы ">" в начале строки, независимо от количества и пробелов
    return re.sub(r'^(>\s*)+', '', text).strip()

# Функция для удаления специфической фразы, если текст начинается с указанного слова
def remove_specific_phrase(text, phrase):
    if text.startswith(phrase.split()[0]):
        # Найдём конец этой фразы
        end_index = text.find(phrase) + len(phrase)
        # Вернём текст после фразы, с учётом возможных специфических фраз
        if phrase.split()[0] == '2.1.':
            return text[end_index+5:].strip()
        else:
            return text[end_index+3:].strip()
    # Если текст не начинается с "Цветом", вернём его без изменений
    return text

# Главная функция для обработки текста и создания массива
def process_text_data(text):
    # Определяем стоп-слова
    stopwords = {'и', 'в', 'по', 'на', 'с', 'также', 'или', 'что', 'но', 'за', 'продавец', 'продавца', 'продавцу', 'продавцом', '.', ',', 'РТТ', '>2.1.', '>>2.1.', '>продавец', '>>продавец'}

    # Фразы, которые нужно удалить
    specific_phrase = "Цветом выделены отклонения выявленные прошлый просмотр, необходимо обратить внимание."
    specific_phrase2 = "2.1. Текущая оценка от "
    result = []
    
    for block in text:
        
        # Удаляем все ведущие символы ">" в начале строки
        block = remove_leading_arrows(block)

        # Извлекаем ФИО, адрес и дату из текста с адаптированными регулярными выражениями
        fio_match = re.search(r'продавцу\s+([\w\s]+)', block, re.IGNORECASE)
        tt_address_match = re.search(r'((?:г\.\s*|п\.\s*)[\w\s]+ул\.[\w\s\d/]+)', block, re.IGNORECASE)
        date_match = re.search(r'дата[:]*\s*([\d.]+)', block, re.IGNORECASE)
        
        fio = fio_converter(fio_match.group(1)) if fio_match else "Не найдено"
        tt_address = tt_address_match.group(1).strip() if tt_address_match else "Не найдено"
        date = date_match.group(1) if date_match else "Не найдено"

        # Очищаем текст
        cleaned_text = multi_sliser(block, ['2.Выявленные отклонения'], 'delete_before')
        cleaned_text = multi_sliser(cleaned_text, ['С уважением'], 'delete_after')
        cleaned_text = multi_sliser(cleaned_text, ['2.2.'], 'delete_after')

        # Удаляем фразы
        cleaned_text = remove_specific_phrase(cleaned_text, specific_phrase)
        cleaned_text = remove_specific_phrase(cleaned_text, specific_phrase2)

        # Удаляем стоп-слова
        cleaned_text = remove_stopwords(cleaned_text, stopwords)
        
        # Удаляем повторяющиеся слова
        cleaned_text = remove_duplicates(cleaned_text)
        cleaned_text = cleaned_text.lower()
        # Добавляем обработанный блок в результат
        result.append([fio, tt_address, date, cleaned_text])

    return result

# Применение функции
# text_data — массив блоков текста
processed_data = process_text_data(text)

# Вывод результата
for entry in processed_data:
    print(entry)


['Хорошилов Михаил Владимирович', 'г.Светлоград ул.Сельскохозяйственная', '17.10.2024', 'когда пришли клиенты, пол был грязным. центре полу лежала метла. не задает открытых вопросов уточняющих режим эксплуатации авто/другого средства мобильности, характеристики авто, сложности эксплуатации, выясняет наличие доп. оборудования (потребителя энергии). использовал способы стимулирования клиента к совершению покупки. показывает состояние уровень жидкости акб. рассказывает/напоминает о режиме предложил клиенту купить сопутствующий ассортимент, аксессуары. не предлагал рассрочку. выдал визитки. сообщил бсо. сообщил, нужно приезжать через 3-4 месяца проверку. поступит звонок от колл-центра.']
['Воронков Сергей Владимирович', 'Не найдено', '17.10.2024', 'не проверил текущее состояние акб. использовал способы стимулирования клиента к совершению покупки. не сообщил, рекомендовано приезжать проверку через 3-4 месяца. прошлая оценка от 12.10 задает вопросов уточняющих режим эксплуатации. показывает 

In [11]:
import pandas as pd
# Создание DataFrame из массива данных
df = pd.DataFrame(processed_data, columns=["ФИО продавца", "Адрес ТТ", "Дата", "Очищенный текст"])
df.drop(0, inplace=True)
# os.remove('data.xlsx')
df.to_excel('data.xlsx')

Первый способ

In [12]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

# Инициализация токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

# Извлечение текстов из столбца
sentences = df['Очищенный текст'].tolist()

# Если столбец 'embeddings' уже существует, удаляем его
if 'embeddings' in df.columns:
    df.drop(columns=['embeddings'], inplace=True)

# Создание списка для сохранения эмбеддингов
embeddings_list = []

# Векторизация предложений с использованием LaBSE
for s in sentences:
    encoded_input = tokenizer(s, padding=True, truncation=True, max_length=64, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embedding = model_output.pooler_output
    embeddings_list.append(embedding[0].numpy())  # Получаем вектор как numpy массив

# Преобразуем список эмбеддингов в массив
embeddings = np.asarray(embeddings_list)
print('\n', len(embeddings), 'x', len(embeddings[0]), '\n')

# Добавляем новые эмбеддинги в датафрейм
df['embeddings'] = embeddings.tolist()
# Сохраняем датафрейм в Excel
df.to_excel('data.xlsx')



 269 x 768 



Второй и третий способ

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import pandas as pd

def vectorize_texts(texts, method="tfidf"):
    if method == "tfidf":
        # Векторизация с помощью TF-IDF (без стоп-слов)
        vectorizer = TfidfVectorizer(max_features=1000)
        X = vectorizer.fit_transform(texts).toarray()  # Преобразуем в плотный массив
        print("Размерность матрицы TF-IDF:", X.shape)
        return X

    elif method == "bert":
        # Векторизация с помощью Sentence-BERT
        model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        X = model.encode(texts)
        print("Размерность матрицы BERT:", X.shape)
        return X
    

# Извлекаем очищенные тексты из датафрейма
cleaned_texts = df['Очищенный текст'].tolist()

# Удаляем столбец embeddings TF-IDF, если он уже существует
if 'embeddings TF-IDF' in df.columns:
    df.drop(columns=['embeddings TF-IDF'], inplace=True)

# Векторизация с использованием TF-IDF
X_tfidf = vectorize_texts(cleaned_texts, method="tfidf")
# Добавляем вектора как новый столбец, каждый вектор как отдельный список
df['embeddings TF-IDF'] = list(X_tfidf)

# Удаляем столбец embeddings Sentence-BERT, если он уже существует
if 'embeddings Sentence-BERT' in df.columns:
    df.drop(columns=['embeddings Sentence-BERT'], inplace=True)

# Векторизация с использованием Sentence-BERT
X_bert = vectorize_texts(cleaned_texts, method="bert")
# Добавляем вектора как новый столбец, каждый вектор как отдельный список
df['embeddings Sentence-BERT'] = list(X_bert)

# Сохраняем датафрейм в Excel
df.to_excel('data.xlsx')


Размерность матрицы TF-IDF: (269, 465)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\sanya\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sanya\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Размерность матрицы BERT: (269, 384)


In [14]:
df

,ФИО продавца,Адрес ТТ,Дата,Очищенный текст,embeddings,embeddings TF-IDF,embeddings Sentence-BERT
1,Воронков Сергей Владимирович,Не найдено,17.10.2024,не проверил текущее состояние акб. использовал...,"[-0.40605172514915466, 0.5676323175430298, -0....","[0.0, 0.2641074362202742, 0.0, 0.3451280682546...","[0.021372376, 0.115748845, 0.028288256, -0.298..."
2,Шульженко Никита Андреевич,Не найдено,14.10.2024,не задает открытых вопросов уточняющих режим э...,"[-0.41750243306159973, -0.012586541473865509, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.270...","[0.042710427, 0.1548902, -0.044505354, -0.1229..."
3,Евланов Андрей Юрьевич,Не найдено,14.10.2024,не проверил текущее состояние акб. задает откр...,"[-0.5012819170951843, 0.012747802771627903, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.292...","[0.03622968, 0.07837429, -0.057751473, -0.1088..."
4,Газюр Иван Александрович,г.Невинномысск ул. Гагарина,14.10.2024,не использовал способы стимулирования клиента ...,"[-0.4312092661857605, 0.41128847002983093, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.006302618, 0.14870247, -0.091361165, -0.340..."
5,Шарафеев Роман Халимович,Не найдено,18.10.2024,находится тт без бейджа. не задает открытых во...,"[-0.48751944303512573, 0.3256247043609619, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.326...","[0.11293991, 0.043835245, 0.008999126, -0.2027..."
...,...,...,...,...,...,...,...
265,Чернышёв Андрей Алексеевич,Не найдено,27.09.2024,находиться ртт не уставной форме одежды обуви....,"[-0.591986894607544, 0.190727099776268, -0.316...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.0801706, 0.17511532, -0.1371665, -0.154168..."
266,Киселёв Денис Николаевич,Не найдено,26.09.2024,не проверил текущее состояние акб. момент заве...,"[-0.4121675491333008, 0.4967484474182129, -0.7...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.095878795, 0.109620534, -0.07306712, -0.27..."
267,Изварин Владимир Павлович,Не найдено,26.09.2024,не использовал способы стимуляции клиента для ...,"[-0.29481032490730286, 0.4810492694377899, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.04345471, 0.116950095, 0.20589969, -0.24657..."
268,Кобрин Владислав Александрович,Не найдено,26.09.2024,не проверил состояние текущего акб. так же про...,"[-0.595455527305603, 0.7615640759468079, -0.88...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.1321206, 0.07315433, -0.07691516, -0.25874..."


In [16]:
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd

# Укажите параметры DBSCAN
dbscan = DBSCAN(eps=0.1, min_samples=5, metric='cosine')

# Применяем DBSCAN для кластеризации
cluster_labels = dbscan.fit_predict(embeddings)

# Добавляем метки кластеров в датафрейм
df['DBSCAN Cluster'] = cluster_labels

# Выводим результат
print("Число кластеров:", len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0))  # -1 — это шум
print(df[['Очищенный текст', 'DBSCAN Cluster']].head())

# Сохраняем результат в Excel
df.to_excel('data.xlsx', index=False)


Число кластеров: 3
                                     Очищенный текст  DBSCAN Cluster
1  не проверил текущее состояние акб. использовал...              -1
2  не задает открытых вопросов уточняющих режим э...               0
3  не проверил текущее состояние акб. задает откр...              -1
4  не использовал способы стимулирования клиента ...               0
5  находится тт без бейджа. не задает открытых во...               0


In [17]:
import plotly.express as px
from sklearn.decomposition import PCA
import umap


reducer = umap.UMAP(n_neighbors=5, min_dist=0.1, n_components=2, random_state=42)
embeddings_2d = reducer.fit_transform(embeddings)

# Добавляем результаты в датафрейм
df['UMAP1'] = embeddings_2d[:, 0]
df['UMAP2'] = embeddings_2d[:, 1]
df['Cluster'] = cluster_labels

# Построение интерактивного scatter plot с Plotly
fig = px.scatter(
    df, x='UMAP1', y='UMAP2',
    color='Cluster',
    hover_data={'Очищенный текст': True, 'UMAP1': False, 'UMAP2': False, 'Cluster': True}
)

# Настройка графика
fig.update_layout(
    title="Интерактивная кластеризация текстов с UMAP и DBSCAN",
    xaxis_title="UMAP1",
    yaxis_title="UMAP2"
)

fig.show()


c:\Users\sanya\anaconda3\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [18]:
import plotly.express as px
import umap
import pandas as pd

# Уменьшение размерности до 2D для визуализации с использованием UMAP
reducer = umap.UMAP(n_neighbors=5, min_dist=0.1, n_components=2, random_state=42)
embeddings_2d = reducer.fit_transform(embeddings)

# Добавляем результаты UMAP в датафрейм
df['UMAP1'] = embeddings_2d[:, 0]
df['UMAP2'] = embeddings_2d[:, 1]
df['Cluster'] = cluster_labels  # предполагаем, что cluster_labels уже определены

# Построение интерактивного scatter plot с Plotly
fig = px.scatter(
    df, x='UMAP1', y='UMAP2',
    color='Cluster',
    hover_data={'Очищенный текст': True, 'UMAP1': False, 'UMAP2': False, 'Cluster': True}
)

# Настройка графика
fig.update_layout(
    title="Интерактивная кластеризация текстов с UMAP и DBSCAN",
    xaxis_title="UMAP1",
    yaxis_title="UMAP2"
)

fig.show()


c:\Users\sanya\anaconda3\Lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

